In [1]:
import numpy as np
import pandas as pd
import requests as requests
import time as time
import hmac
import hashlib
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df= pd.read_csv('BTC_with_Gold_USD_minute.csv')

In [3]:
def FeatureEngineering(df):
    import numpy as np
    import pandas as pd
    import requests as requests
    import time as time
    import hmac
    import hashlib
    import os
    from datetime import datetime, timedelta
    from dotenv import load_dotenv
    import json
    import matplotlib.pyplot as plt
    import seaborn as sns
    df["btc_return"] = df["close"].pct_change()
    df["gold_return"] = df["gold_close_gc=f"].pct_change()
    df["usd_return"] = df["usd_close_dx=f"].pct_change()
    window = 60
    df["btc_momentum"] = df["btc_return"].rolling(window).mean()
    df["btc_volatility"] = df["btc_return"].rolling(window).std()
    df["btc_volume_mean"] = df["volume"].rolling(window).mean()

    df["gold_momentum"] = df["gold_return"].rolling(window).mean()
    df["gold_volatility"] = df["gold_return"].rolling(window).std()

    df["usd_momentum"] = df["usd_return"].rolling(window).mean()
    df["usd_volatility"] = df["usd_return"].rolling(window).std()
    df["target"] = df["btc_return"].shift(-1)
    drop_cols = [
    "open", "high", "low", "close", "volume",
    "gold_open_gc=f", "gold_high_gc=f", "gold_low_gc=f", "gold_close_gc=f", "gold_volume_gc=f",
    "usd_open_dx=f", "usd_high_dx=f", "usd_low_dx=f", "usd_close_dx=f", "usd_volume_dx=f"
    ]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])
    cols = ['btc_return', 'gold_return', 'usd_return', 'btc_momentum',
       'btc_volatility', 'btc_volume_mean', 'gold_momentum', 'gold_volatility',
       'usd_momentum', 'usd_volatility', 'target', 'time']
    df = df.copy()
    df[cols] = (df[cols]
                .replace(["", "nan", "NaN"], pd.NA)
                .ffill()
                .bfill()) 
    for lag in [1, 2, 3, 6, 12, 24]:
        df[f'btc_return_lag_{lag}'] = df['btc_return'].shift(lag)
        df[f'btc_volatility_lag_{lag}'] = df['btc_volatility'].shift(lag)

    windows = [3, 6, 12, 24]
    for w in windows:
        df[f'btc_return_rolling_mean_{w}'] = df['btc_return'].rolling(w).mean()
        df[f'btc_return_rolling_std_{w}'] = df['btc_return'].rolling(w).std()
        df[f'btc_momentum_rolling_mean_{w}'] = df['btc_momentum'].rolling(w).mean()
    
    df['btc_gold_corr_6h'] = df['btc_return'].rolling(6).corr(df['gold_return'])
    df['btc_usd_corr_6h'] = df['btc_return'].rolling(6).corr(df['usd_return'])
    df['btc_gold_spread'] = df['btc_return'] - df['gold_return']
    df['btc_usd_spread'] = df['btc_return'] - df['usd_return']
    df['btc_gold_momentum_diff'] = df['btc_momentum'] - df['gold_momentum']
    df['btc_usd_momentum_diff'] = df['btc_momentum'] - df['usd_momentum']
    df['btc_volatility_sqrt'] = np.sqrt(df['btc_volatility'])
    df['btc_momentum_sq'] = df['btc_momentum'] ** 2
    df['log_btc_volume'] = np.log1p(df['btc_volume_mean'])
    df['vol_mom_ratio'] = df['btc_momentum'] / (df['btc_volatility'] + 1e-6)
    df['risk_on'] = ((df['gold_return'] < 0) & (df['btc_return'] > 0)).astype(int)
    df['risk_off'] = ((df['gold_return'] > 0) & (df['btc_return'] < 0)).astype(int)

    corr_matrix = df.drop(columns=['time']).corr().abs()
    high_corr_features = set()

    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if (corr_matrix.iloc[i, j] > 0.95):
                high_corr_features.add(corr_matrix.columns[j])
    features_to_drop = list(high_corr_features)
    df_cleaned = df.drop(columns=features_to_drop)
    final_df=df_cleaned
    final_df['target'] =   final_df['btc_return'].rolling(12).std().shift(-12)
    final_df['high_vol_regime'] = (final_df['btc_volatility'] > final_df['btc_volatility'].rolling(24).mean()).astype(int)
    final_df['trend_regime'] = (final_df['btc_return_rolling_mean_24'] > 0).astype(int)
    cols = ['btc_return', 'gold_return', 'usd_return', 'btc_momentum',
       'btc_volatility', 'btc_volume_mean', 'gold_momentum', 'gold_volatility',
       'usd_momentum', 'usd_volatility', 'target', 'time',
       'btc_return_lag_1', 'btc_return_lag_2', 'btc_return_lag_3',
       'btc_return_lag_6', 'btc_return_lag_12', 'btc_volatility_lag_12',
       'btc_return_lag_24', 'btc_volatility_lag_24',
       'btc_return_rolling_mean_3', 'btc_return_rolling_std_3',
       'btc_return_rolling_mean_6', 'btc_return_rolling_std_6',
       'btc_return_rolling_mean_12', 'btc_return_rolling_std_12',
       'btc_return_rolling_mean_24', 'btc_return_rolling_std_24',
       'btc_gold_corr_6h', 'btc_usd_corr_6h', 'btc_gold_spread',
       'btc_gold_momentum_diff', 'btc_volatility_sqrt', 'btc_momentum_sq',
       'log_btc_volume', 'vol_mom_ratio', 'risk_on', 'risk_off']
    final_df = final_df.copy()
    final_df[cols] = (final_df[cols]
                .replace(["", "nan", "NaN"], pd.NA)
                .ffill()
                .bfill())
    return final_df


    


In [4]:
new=FeatureEngineering(df)

In [5]:
new.shape

(104000, 40)

In [35]:
new.describe()

,btc_return,gold_return,usd_return,btc_momentum,btc_volatility,btc_volume_mean,gold_momentum,gold_volatility,usd_momentum,usd_volatility,...,btc_gold_spread,btc_gold_momentum_diff,btc_volatility_sqrt,btc_momentum_sq,log_btc_volume,vol_mom_ratio,risk_on,risk_off,high_vol_regime,trend_regime
count,103939.000000,103939.000000,1.039390e+05,103939.000000,103939.000000,103939.000000,103939.000000,103939.000000,1.039390e+05,103939.000000,...,1.039390e+05,1.039390e+05,103939.000000,1.039390e+05,103939.000000,103939.000000,103939.000000,103939.000000,103939.000000,103939.000000
mean,0.000003,0.000002,6.793157e-08,0.000003,0.000496,104.360889,0.000002,0.000228,6.438787e-08,0.000086,...,7.244076e-07,6.709043e-07,0.021314,7.532203e-09,4.432803,0.003745,0.003868,0.003954,0.480397,0.499966
std,0.000704,0.000409,1.535056e-04,0.000087,0.000500,85.697500,0.000053,0.000341,1.975428e-05,0.000127,...,8.398243e-04,1.052513e-04,0.006491,6.024553e-08,0.647483,0.116427,0.062070,0.062759,0.499618,0.500002
min,-0.079065,-0.021819,-1.282572e-02,-0.001381,0.000000,0.000000,-0.000373,0.000000,-2.154527e-04,0.000000,...,-8.840017e-02,-1.536115e-03,0.000000,0.000000e+00,0.000000,-0.430970,0.000000,0.000000,0.000000,0.000000
25%,-0.000236,0.000000,0.000000e+00,-0.000031,0.000312,52.833333,-0.000016,0.000058,-6.761137e-06,0.000019,...,-2.400467e-04,-3.673510e-05,0.017656,2.244773e-10,3.985893,-0.078809,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000e+00,0.000001,0.000407,77.383333,0.000000,0.000141,0.000000e+00,0.000053,...,0.000000e+00,8.411382e-07,0.020174,1.043029e-09,4.361611,0.003634,0.000000,0.000000,0.000000,0.000000
75%,0.000239,0.000000,0.000000e+00,0.000034,0.000549,124.116667,0.000020,0.000271,7.600785e-06,0.000111,...,2.418026e-04,4.018433e-05,0.023438,3.515789e-09,4.829247,0.084578,0.000000,0.000000,1.000000,1.000000
max,0.065479,0.037524,1.380238e-02,0.001122,0.010236,925.966667,0.000625,0.004854,2.376594e-04,0.001785,...,7.445151e-02,1.298026e-03,0.101173,1.905849e-06,6.831918,0.514209,1.000000,1.000000,1.000000,1.000000


In [36]:
df['gold_return']

60        NaN
61        0.0
62        0.0
63        0.0
64        0.0
         ... 
103994    0.0
103995    0.0
103996    0.0
103997    0.0
103998    0.0
Name: gold_return, Length: 103939, dtype: float64

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass 
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        X = X.copy()
        X = FeatureEngineering(X)  
        return X


In [5]:
from sklearn.preprocessing import RobustScaler
from scipy.stats.mstats import winsorize
from statsmodels.graphics.tsaplots import plot_acf
def apply_scaler(final_df, scaler=None, fit=False):
    scaler = RobustScaler()
    scaled = scaler.fit_transform(final_df.select_dtypes(include=np.number))
    final_df_scaled = pd.DataFrame(scaled, columns=final_df.select_dtypes(include=np.number).columns)
    def cap_outliers(df, cols=None, lower=0.01, upper=0.99):
        df_capped = df.copy()
        if cols is None:
            cols = df.select_dtypes(include=np.number).columns
        for col in cols:
            lower_val = df_capped[col].quantile(lower)
            upper_val = df_capped[col].quantile(upper)
            df_capped[col] = np.clip(df_capped[col], lower_val, upper_val)
        return df_capped
    df_capped = cap_outliers(final_df_scaled, lower=0.02, upper=0.98)
    for col in df_capped.columns:
        if df_capped[col].dtype != 'object':  
            df_capped[col] = winsorize(df_capped[col], limits=[0.05, 0.05])
    scaler = RobustScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df_capped), columns=df_capped.columns)
    for col in ['btc_volume_mean', 'btc_volatility', 'btc_momentum_sq', 'btc_volatility_sqrt']:
        df_capped[col] = np.log1p(df_capped[col])
    return df_capped,scaler 

In [6]:
class CustomScaler(BaseEstimator, TransformerMixin):
    import numpy as np
    import pandas as pd
    from scipy.stats.mstats import winsorize
    from sklearn.base import BaseEstimator, TransformerMixin
    from sklearn.preprocessing import RobustScaler
    def __init__(self, lower=0.02, upper=0.98, winsorize_limits=(0.05, 0.05), log_features=None):
        self.lower = lower
        self.upper = upper
        self.winsorize_limits = winsorize_limits
        self.log_features = log_features or ['btc_volume_mean', 'btc_volatility', 'btc_momentum_sq', 'btc_volatility_sqrt']
        self.scaler = RobustScaler()
    
    def fit(self, X, y=None):
        X_num = X.select_dtypes(include=np.number)
        X_capped = self._cap_outliers(X_num)
        X_wins = self._apply_winsorize(X_capped)
        self.scaler.fit(X_wins)
        return self
    
    def transform(self, X):
        X_num = X.select_dtypes(include=np.number)
        X_capped = self._cap_outliers(X_num)
        X_wins = self._apply_winsorize(X_capped)
        X_scaled = pd.DataFrame(self.scaler.transform(X_wins), columns=X_wins.columns, index=X_wins.index)
        
        for col in self.log_features:
            if col in X_scaled.columns:
                X_scaled[col] = np.log1p(X_scaled[col].clip(lower=0))
        
        X_rest = X.select_dtypes(exclude=np.number)
        X_final = pd.concat([X_scaled, X_rest], axis=1)
        
        return X_final
    
    def _cap_outliers(self, df):
        df_capped = df.copy()
        for col in df.columns:
            lower_val = df_capped[col].quantile(self.lower)
            upper_val = df_capped[col].quantile(self.upper)
            df_capped[col] = np.clip(df_capped[col], lower_val, upper_val)
        return df_capped
    
    def _apply_winsorize(self, df):
        df_wins = df.copy()
        for col in df.columns:
            df_wins[col] = winsorize(df_wins[col], limits=self.winsorize_limits)
        return df_wins


In [7]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('features', FeatureEngineer()),   
    ('scaler', CustomScaler())         
])


In [8]:
from sklearn.model_selection import TimeSeriesSplit
def get_time_series_splits(X, y, n_splits=3):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = []
    
    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        splits.append((X_train, X_val, y_train, y_val))
    
    return splits


In [9]:
X = FeatureEngineering(df)
X = CustomScaler().fit_transform(X)
y = X['target']
X = X.drop(columns=['target'])
splits = get_time_series_splits(X, y, n_splits=3)


In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor   
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'ExtraTrees': ExtraTreesRegressor(random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'XGB': XGBRegressor(random_state=42, n_jobs=-1),
    'LGBM': LGBMRegressor(random_state=42, n_jobs=-1)
}
param_grids = {
    'DecisionTree': {'max_depth':[5,10,15], 'min_samples_split':[2,5,10], 'min_samples_leaf':[1,3,5]},
    'RandomForest': {'n_estimators':[100,200], 'max_depth':[None,10,20], 'min_samples_split':[2,5,10], 'min_samples_leaf':[1,3,5]},
    'ExtraTrees': {'n_estimators':[100,200], 'max_depth':[None,10,20], 'min_samples_split':[2,5,10]},
    'AdaBoost': {'n_estimators':[50,100,200], 'learning_rate':[0.01,0.05,0.1]},
    'GradientBoosting': {'n_estimators':[100,200], 'learning_rate':[0.01,0.05,0.1], 'max_depth':[3,4,5]},
    'XGB': {'n_estimators':[100,200,300], 'learning_rate':[0.01,0.05,0.1], 'max_depth':[4,6,8]},
    'LGBM': {'n_estimators':[100,200,300], 'learning_rate':[0.01,0.05,0.1], 'num_leaves':[31,50,70]}
}

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score

def tune_models(X, y, models, param_grids, n_iter=15, n_splits=3, scoring='r2'):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    best_models = {}

    for name, model in models.items():
        print(f"\n🔹 Tuning {name}...")
        if name not in param_grids:
            print(f"⚠️ No param grid found for {name}, skipping...")
            continue

        search = RandomizedSearchCV(
            model,
            param_distributions=param_grids[name],
            n_iter=n_iter,
            cv=tscv,
            scoring=scoring,
            n_jobs=-1,
            random_state=42,
            refit=True,
            verbose=1
        )

        search.fit(X, y)
        best_models[name] = search.best_estimator_

        results.append({
            'Model': name,
            'Best Score': search.best_score_,
            'Best Params': search.best_params_
        })

    results_df = pd.DataFrame(results).sort_values(by='Best Score', ascending=False)
    return results_df, best_models


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# -----------------------------
# Data cleaning and preparation
# -----------------------------
df = df.dropna(subset=['target'])
df = df.replace([np.inf, -np.inf], np.nan).dropna()

X = df.drop(columns=['target', 'time']).values
y = df['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# -----------------------------
# Neural Network Tuning Function
# -----------------------------
def build_model(hp):
    model = keras.Sequential([
        layers.Dense(
            hp.Int('units_input', 64, 256, step=32),
            activation='relu',
            input_shape=(X_train.shape[1],)
        ),
        layers.BatchNormalization(),
        layers.Dropout(hp.Float('dropout_input', 0.1, 0.5, step=0.1)),

        layers.Dense(
            hp.Int('units_hidden1', 32, 128, step=32),
            activation='relu'
        ),
        layers.BatchNormalization(),
        layers.Dropout(hp.Float('dropout_hidden1', 0.1, 0.5, step=0.1)),

        layers.Dense(
            hp.Int('units_hidden2', 32, 128, step=32),
            activation='relu'
        ),
        layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        ),
        loss='mse',
        metrics=['mae']
    )
    return model

# -----------------------------
# Keras Tuner Search
# -----------------------------
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=15,
    executions_per_trial=1,
    directory='my_tuner_dir',
    project_name='volatility_nn_tuning'
)

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
    verbose=1
)

# -----------------------------
# Best Model Training
# -----------------------------
best_hp = tuner.get_best_hyperparameters(1)[0]
print("\nBest Hyperparameters:")
for k, v in best_hp.values.items():
    print(f"{k}: {v}")

best_model = tuner.hypermodel.build(best_hp)
best_model.fit(
    X_scaled, y,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    verbose=1
)

# -----------------------------
# Evaluation
# -----------------------------
y_pred_nn = best_model.predict(X_val).flatten()
y_pred_nn = np.nan_to_num(y_pred_nn)
r2_nn = r2_score(y_val, y_pred_nn)

# -----------------------------
# Combine results with other models
# -----------------------------
# If results_df from your sklearn models exists, use it
# Otherwise, initialize a new one
try:
    results_df
except NameError:
    results_df = pd.DataFrame(columns=['Model', 'R2_Score'])

new_row = pd.DataFrame([{'Model': 'NeuralNetwork_Tuned', 'R2_Score': r2_nn}])
results_df = pd.concat([results_df, new_row], ignore_index=True)

print("\n📊 Model Performance Comparison:")
print(results_df.sort_values(by='R2_Score', ascending=False))


Reloading Tuner from my_tuner_dir\volatility_nn_tuning\tuner0.json

Best Hyperparameters:
units_input: 192
dropout_input: 0.30000000000000004
units_hidden1: 96
dropout_hidden1: 0.4
units_hidden2: 64
learning_rate: 0.000791524243562959
Epoch 1/100


c:\Users\naman\OneDrive\Desktop\delta_Exchange_india\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2599/2599 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0597 - mae: 0.1408 - val_loss: 0.0015 - val_mae: 0.0285
Epoch 2/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 5.6190e-04 - mae: 0.0153 - val_loss: 1.4983e-05 - val_mae: 0.0029
Epoch 3/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3.0120e-06 - mae: 9.9343e-04 - val_loss: 8.5993e-07 - val_mae: 3.8041e-04
Epoch 4/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 2.1404e-06 - mae: 9.2639e-04 - val_loss: 2.4630e-06 - val_mae: 0.0012
Epoch 5/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 4.8657e-06 - mae: 0.0014 - val_loss: 2.2057e-05 - val_mae: 0.0039
Epoch 6/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 4.6320e-06 - mae: 0.0014 - val_loss: 1.2523e-06 - val_mae: 6.3673e-04
Epoch 7/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 2.4709e-06 - mae: 0.0011 - val_loss: 1.3997e-06 - val_mae: 7.4086e-04
Epoch 8/100
2599/2599 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1.3721e-06 - mae: 8.0148e-04 - v

C:\Users\naman\AppData\Local\Temp\ipykernel_8864\3719458086.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow import keras
import keras_tuner as kt
import joblib

class ModelSelectionPipeline:
    def __init__(self, models, param_grids, scaler=None, cv_splits=5):
        self.models = models
        self.param_grids = param_grids
        self.scaler = scaler or StandardScaler()
        self.cv = TimeSeriesSplit(n_splits=cv_splits)
        self.results = []
        self.best_models = {}

    def preprocess(self, X):
        return self.scaler.fit_transform(X)
    
    def tune_sklearn_models(self, X, y):
        for name, model in self.models.items():
            print(f"Tuning {name}...")
            search = RandomizedSearchCV(
                model,
                param_distributions=self.param_grids[name],
                n_iter=15,
                cv=self.cv,
                scoring='r2',
                n_jobs=-1,
                random_state=42,
                refit=True,
                verbose=0
            )
            search.fit(X, y)
            self.best_models[name] = search.best_estimator_
            self.results.append({
                'Model': name,
                'Best Score': search.best_score_,
                'Best Params': search.best_params_
            })

    def tune_neural_network(self, X_train, X_val, y_train, y_val, build_model_fn):
        tuner = kt.RandomSearch(
            build_model_fn,
            objective='val_mae',
            max_trials=15,
            executions_per_trial=1,
            directory='tuner_results',
            project_name='nn_tuning'
        )

        tuner.search(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=32,
            callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
            verbose=0
        )

        best_hp = tuner.get_best_hyperparameters(1)[0]
        best_nn = build_model_fn(best_hp)
        best_nn.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=32,
            callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
            verbose=0
        )

        y_pred = best_nn.predict(X_val).flatten()
        r2_nn = r2_score(y_val, y_pred)
        self.best_models['NeuralNetwork'] = best_nn
        self.results.append({'Model': 'NeuralNetwork', 'Best Score': r2_nn, 'Best Params': best_hp.values})

    def get_results(self):
        df = pd.DataFrame(self.results).sort_values(by='Best Score', ascending=False)
        self.best_model_name = df.iloc[0]['Model']
        print(df)
        print(f"\nBest model overall: {self.best_model_name}")
        return df

    def save_best_model(self):
        best_model = self.best_models[self.best_model_name]
        if self.best_model_name == 'NeuralNetwork':
            best_model.save('best_model.h5')
        else:
            joblib.dump(best_model, 'best_model.pkl')
        print("✅ Model saved successfully.")


In [19]:
pipeline = ModelSelectionPipeline(models, param_grids)
X_scaled = pipeline.preprocess(X)
pipeline.tune_sklearn_models(X_scaled, y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
pipeline.tune_neural_network(X_train, X_val, y_train, y_val, build_model)
results_df = pipeline.get_results()
pipeline.save_best_model()

Tuning DecisionTree...
Tuning RandomForest...
Tuning ExtraTrees...
Tuning AdaBoost...


c:\Users\naman\OneDrive\Desktop\delta_Exchange_india\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=15. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Tuning GradientBoosting...
Tuning XGB...
Tuning LGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006538 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 103939, number of used features: 25
[LightGBM] [Info] Start training from score 0.000003


c:\Users\naman\OneDrive\Desktop\delta_Exchange_india\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              Model  Best Score  \
3          AdaBoost   -0.000267   
7     NeuralNetwork   -0.002166   
6              LGBM   -0.022943   
5               XGB   -0.026073   
0      DecisionTree   -0.144092   
2        ExtraTrees   -0.607332   
1      RandomForest   -0.652553   
4  GradientBoosting   -1.392748   

                                         Best Params  
3        {'n_estimators': 50, 'learning_rate': 0.01}  
7  {'units_input': 224, 'dropout_input': 0.4, 'un...  
6  {'num_leaves': 31, 'n_estimators': 100, 'learn...  
5  {'n_estimators': 100, 'max_depth': 4, 'learnin...  
0  {'min_samples_split': 10, 'min_samples_leaf': ...  
2  {'n_estimators': 200, 'min_samples_split': 10,...  
1  {'n_estimators': 100, 'min_samples_split': 5, ...  
4  {'n_estimators': 100, 'max_depth': 3, 'learnin...  

Best model overall: AdaBoost
✅ Model saved successfully.


In [21]:
if pipeline.best_model_name == 'NeuralNetwork':
    best_model = keras.models.load_model('best_model.h5')
else:
    best_model = joblib.load('best_model.pkl')
X_new_scaled = pipeline.scaler.transform(X_new)
y_pred = best_model.predict(X_new_scaled)


NameError: name 'X_new' is not defined

In [22]:
results_df

,Model,Best Score,Best Params
3,AdaBoost,-0.000267,"{'n_estimators': 50, 'learning_rate': 0.01}"
7,NeuralNetwork,-0.002166,"{'units_input': 224, 'dropout_input': 0.4, 'un..."
6,LGBM,-0.022943,"{'num_leaves': 31, 'n_estimators': 100, 'learn..."
5,XGB,-0.026073,"{'n_estimators': 100, 'max_depth': 4, 'learnin..."
0,DecisionTree,-0.144092,"{'min_samples_split': 10, 'min_samples_leaf': ..."
2,ExtraTrees,-0.607332,"{'n_estimators': 200, 'min_samples_split': 10,..."
1,RandomForest,-0.652553,"{'n_estimators': 100, 'min_samples_split': 5, ..."
4,GradientBoosting,-1.392748,"{'n_estimators': 100, 'max_depth': 3, 'learnin..."


In [ ]:
final_d

NameError: name 'final_df' is not defined